<a href="https://colab.research.google.com/github/astorguy/learn_ngspice/blob/main/notebooks/divider/divider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Example 1: Introduction and First Ngspice Simulation***

## Introduction
This is a hands-on introduction to [Ngspice](https://ngspice.sourceforge.io/), using [Google Colaboratory](https://colab.research.google.com/). A "Colab" is essentially a [Jupyter](https://jupyter.org/) notebook in the Google ecosystem. Jupyter notesbooks interactively generate, process, and visualize data using Python. We will be using the [py4spice](https://pypi.org/project/py4spice/) package to facilitate interaction with Ngspice.

Our objective is to learn how to use Ngspice for circuit simulation and analysis. We'll going through progressively more complicated circuits, analyses, and data processing, explaining Ngspice concepts along the way.

This notebook can be run in Google Colab or in Jupyter. However, the setup is slightly different. This function determines which environment we are using.

In [7]:
def running_in_colab() -> bool:
    try:
        import google.colab  # type: ignore
        return True
    except ImportError:
        return False

## Installations and Imports


In [8]:
def running_in_colab() -> bool:
    '''Check if the code is running in Google Colab environment'''
    try:
        import google.colab  # type: ignore
        return True
    except ImportError:
        return False

Running in Colabs doesn't require `sudo apt-get update` which takes a bit of time, so we check.

In [9]:
import subprocess

if not running_in_colab():
    subprocess.run(["sudo", "apt-get", "update"], check=True)
subprocess.run(["sudo", "apt-get", "install", "-y", "ngspice"], check=True)

Hit:1 http://deb.debian.org/debian trixie InRelease
Hit:2 http://deb.debian.org/debian trixie-updates InRelease
Hit:3 http://deb.debian.org/debian-security trixie-security InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
ngspice is already the newest version (44.2+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


CompletedProcess(args=['sudo', 'apt-get', 'install', '-y', 'ngspice'], returncode=0)

After installs, import the Python packages we need

In [10]:
from pathlib import Path
import py4spice

## First Ngspice Simulation
Eventually, we will execute Ngspice from Python. But for this first example we will start by running it directly from the command line. This is the traditional method for running Ngspice. However, it's tedious and error-prone, so we'll introduce Python and py4spice in subsequent examples.

## The `divider.cir` Netlist
Ngspice requires a file called a *netlist*. This file contains should have at least the following items:
- The first line ***must*** be the title line. It is treated as a comment
- The last line ***must*** be: `.end`
- In the most general cases, individual components of the circuit are each listed on their own lines.
- The type of simulation is specified with a line starting with a `.` (period)

There are many other aspects to netlists, which we'll cover as needed.

### Circuit components
Every component in the circuit starts with a letter (note that Ngspice is generally case insensitive). Our circuit has three components.
- Any line that begins with an `R` is a resistor. The first word must have one or more additional characters to make it unique. In the first case we use a `1` to make it `R1`. The next words are the connection nodes. For a resistor there are two nodes. The last word is the resistor value. In the case of `R1`, the value is `1k` ohms. For `R1` the connections are `in` and `out`. For `R2` the connections are `out` and `0`, and the value is `2k` ohms.
- Any line that begins with a `V` is a voltage source. Again the first word needs one or more characters to make it unique. The second word is the positive connection node. The third word is the negative connection node. The remaining words describe the type of voltage source, with many different options. In our case, we have a DC voltage source of `10` volts.

### Simulations specification
There are several different types of simulations available in Ngspice. For our simple example, we are performing an "operating point" (`.op`).

The following code creates our netlist and writes it into a file named `divider.cir`.

***Note:*** *Netlists are normally generated directly from a schematic capture, such as [KiCad](https://www.kicad.org/).*

In [11]:
content = """
* First line is always the title
V1 in 0 DC 10
R1 in out 1k
R2 out 0 2k
.op
.end
""".strip()

Path('divider.cir').write_text(content)

80

The following Ngspice command has the `-b` switch for "batch mode" and the name of the file to read in.

In [12]:
subprocess.run(["ngspice", "-b", "divider.cir"], check=True)


Note: No compatibility mode selected!


Circuit: * first line is always the title

Doing analysis at TEMP = 27.000000 and TNOM = 27.000000

Using SPARSE 1.3 as Direct Linear Solver

No. of Data Rows : 1
	Node                                  Voltage
	----                                  -------
	----	-------
	out                              6.666667e+00
	in                               1.000000e+01

	Source	Current
	------	-------

	v1#branch                        -3.33333e-03

 Resistor models (Simple linear resistor)
      model                     R

        rsh                     0
     narrow                     0
      short                     0
        tc1                     0
        tc2                     0
        tce                     0
       defw                 1e-05
          l                 1e-05
         kf                     0
         af                     0
          r                     0
     bv_max                 1e+99
         lf                

CompletedProcess(args=['ngspice', '-b', 'divider.cir'], returncode=0)

Ngspice generates an overload of information. However, if we look closely at the output log, we can see on about line 13 the voltage at `out` is `6.666667e+00` volts.

We have completed our first Ngspice simulation!

Before we move to our next example, let's clean up the directory with the following command:

In [13]:
!rm divider.cir; ls -la

total 48
drwxrwxrwx+ 4 vscode root    4096 Feb  4 21:36 .
drwxrwxrwx+ 3 vscode root    4096 Feb  4 19:34 ..
-rw-rw-rw-  1 vscode root   28980 Feb  4 21:35 divider.ipynb
drwxrwxrwx+ 2 vscode vscode  4096 Feb  4 20:51 netlists
drwxrwxrwx+ 2 vscode vscode  4096 Feb  4 20:57 results


### ***Example 2: Integrating Ngspice into Python with py4spice***

## Introduction
We are going to simulate the same divider circuit, but this time control Ngspice with Python using the py4spice package. At first, this will seem like a more complicated way to use Ngspice. However, it gives us complete control within Python, enabling us to vary parameters, optimize, and generate innovative data presentations.

When using py4spice, Ngspice is run in batch mode, reading a single top-level netlist. The netlist contains everything Ngspice needs for simulation.

When using py4spice the script will typically entail six steps:

1. Set up
2. Create control section
3. Contruct top netlist
4. Execute Ngspice simulation
5. Convert simulation results to [NumPy](https://numpy.org/) arrays
6. Analyze the data and display

More advanced analysis may involve looping through steps 2 through 6 while varying component values or circuit topology.

### Paths dictionary
There are paths and files to define. We're put them into a dictionary to keep things organized. Two subdirectories are also defined:
- /content/netlists
- /content/results

In [14]:
paths = {}  # define empty paths dictionary
paths["ngspice"] = Path("/usr/bin/ngspice") # ngspice executable

paths["workspace"] = Path.cwd()
paths["netlists"] = paths["workspace"] / "netlists"
paths["netlists"].mkdir(parents=True, exist_ok=True) # create netlist directory if does not exist
paths["results"] = paths["workspace"] / "results"
paths["results"].mkdir(parents=True, exist_ok=True) # create results directory if does not exist

We need to create a file for the simulation transcript.

In [15]:
# create simulation transcript file. If it exists, make sure it is empty
paths["sim_transcript"] = paths["results"] / "sim_transcript.log"
if paths["sim_transcript"].exists():  # delete and recreate. this makes sure it's empty
    paths["sim_transcript"].unlink()
paths["sim_transcript"].touch()

### Netlists dictionary
Netlist objects are created. These netlist snippets will be used later to create a single top-level netlist which will be read in during the batch simulation. Again, we put them into a dictionary to keep things organized.

In [16]:
netlists = {}  # define empty netlists dictionary

# create these lines to use when concatenating to make the top netlist
netlists["blankline"] = py4spice.Netlist("")  # blank line for spacing
netlists["title"] = py4spice.Netlist("* Simple Voltage Divider")
netlists["end_line"] = py4spice.Netlist(".end")

netlists["stimulus"] = py4spice.Netlist("vin in 0 dc 10")

### Creating DUT (Device Under Test)
Though this circuit is simple, we are going to separate the DUT from all the other parts of the netlist. The stimulus, simulations, title, and `.end` lines will be added separately.

The netlist for the DUT would normally be created by a schematic capture such as [KiCad](https://www.kicad.org/) and uploaded to the netlists directory.

In [17]:
content = """
R1 in out 1k
R2 out 0 2k
""".strip()

netlists["dut"] = py4spice.Netlist(content)

### Vectors dictionary
Vectors objects are used to define a set of circuit signals to create and display. Again, we put them into a dictionary to keep things organized.

In [18]:
vectors = {}   # start with empty vectors dictionary
vectors["in"] = py4spice.Vectors("in")
vectors["out"] = py4spice.Vectors("out")
vectors["in_out"] = vectors["in"] + vectors["out"]

### Analyses to run during the simulation process
Each batch simulation in Ngspice can perform one or more analyses. In this simple example, we will only be simulating a single analysis. However, we will create a list of analyses to make it easier to reuse the code. The Analysis object has the following arguments:

| Argument | Description |
| :--- | :--- |
| **name** | Label given to the object. |
| **cmd_type** | Ngspice has different types of analyses. The `op`, `tran`, `ac`, and `dc` analyses are supported with `py4spice`. |
| **cmd** | This is the actual command that will be sent to Ngspice. Its arguments can be found in the Ngspice documentation. |
| **vector** | This defines which signals will be in the simulation results. |
| **results_loc** | Where to put the results files. |

In [19]:
list_of_analyses = []  # start with an empty list

# 1st analysis: operating point
op1 = py4spice.Analyses(
    name="op1",
    cmd_type="op",
    cmd="op",
    vector=vectors["in_out"],
    results_loc=paths["results"],
)
list_of_analyses.append(op1)

# For this example there's only one analysis.
#   If there were more, we would define & append.

### Control section
This is the section that tells Ngspice what operations to do. A netlist object snippet is created. It will be concatenated with the other netlist objects to create the top-level netlist.

In [20]:
my_control = py4spice.Control()  # create 'my_control' object

# add all the analyses defined above into the control section
for analysis in list_of_analyses:
    my_control.insert_lines(analysis.lines_for_cntl())

# convert control section into a netlist object
netlists["control1"] = py4spice.Netlist(str(my_control))

### Create a single netlist, ready for simulation
The netlist snippets are concatenated and merged into `top`. `top.cir` is written out. It will be read in during the simulation. We print out `top` to verify everything is correct.

In [21]:
# concatenate all tne netlists to make top1 and add to netlist dict
netlists["top"] = (
    netlists["title"]
    + netlists["blankline"]
    + netlists["stimulus"]
    + netlists["dut"]
    + netlists["blankline"]
    + netlists["control1"]
    + netlists["blankline"]
    + netlists["end_line"]
    + netlists["blankline"]
    )

# write netlist to a file so ngspice can read it
paths["top"] = paths["netlists"] / "top.cir"
netlists["top"].write_to_file(paths["top"])

# here is the netlist in top.cir and will be used for simulation
print(netlists["top"])

* simple voltage divider

vin in 0 dc 10
r1 in out 1k
r2 out 0 2k

.control
* timestamp: wed feb  4 21:36:11 2026
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
op
print line out in > /workspaces/learn_ngspice/notebooks/divider/results/op1.txt
quit
.endc

.end



### Simulate
The Ngspice command is constructed, printed out for verification, and executed. `timeout` (in seconds) is set in case the simulation hangs.

In [22]:
# prepare simulate object and simulate
sim = py4spice.Simulate(
    ngspice_exe=paths["ngspice"],
    netlist_filename=paths["top"],
    transcript_filename=paths["sim_transcript"],
    name="sim1",
    timeout=5,
)
# print out the command that will be used in the spawned process
print(sim.ngspice_command)

sim.run()  # run the Ngspice simulation

['/usr/bin/ngspice', '-b', '/workspaces/learn_ngspice/notebooks/divider/netlists/top.cir']


### Convert simulation results to usable format
Ngspice simulation results are written out to a text file. These results are converted to a `SimResults` object for easier post-processing.

In [23]:
# There's a list of sim_results in case there was more than one analysis for the simulation.
#   in this case there is only one analysis performed during simulation
sim_results = [
    py4spice.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses
]

### Print out operating point (`op`) results
The print_section function creates an easy-to-read table

In [24]:
# display results for operating point analysis
py4spice.print_section("Operating Point Results", sim_results[0].table_for_print())


--- Operating Point Results ---
out  6.667
in   10.000

-------------------------------



## Final notes
This was a very simple circuit, with only a table of values and no post-processing. Other examples will be more complex. However, all of the scripts using py4spice will similarly follow these steps.